__1 - Importing needed dependencies__

In [14]:
import tensorflow as tf
import keras
import os
import sys
import re
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from keras_transformer import get_model, decode
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.models import load_model

__2 - Declaring global variables__

In [15]:
EPOCHS = 10
SENTENCES_MAX_LENGTH = 30
BATCH_SIZE = 128
EMBEDDING_DIM = 128
HIDDEN_DIM = 1024
NUM_TRAIN_PLOTS = 10000

__3 - Try read data__

In [18]:
def clean(text):
    '''
    '''
    text = text.strip()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace(",", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("-", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.lower()
    text = ''.join([i for i in text if not i.isdigit()])

    return text

try:
    dirname = os.path.abspath('')
    filepath = os.path.join(dirname, 'input_data/wiki_movie_plots.csv')
    dataframe = pd.read_csv(filepath, sep=',')
    plotsList = dataframe['Plot']
    print('{} plots imported.'.format(len(plotsList)))
    plotsList = plotsList[:NUM_TRAIN_PLOTS]
    for idx, p in enumerate(plotsList):
        plotsList[idx] = clean(p)
    trainPlotsList = plotsList[:NUM_TRAIN_PLOTS]    

except IOError:
    sys.exit('Cannot find data!')

34886 plots imported.


__4 - Extract vocabulary__

In [19]:
# CREATE VOCABULARY OF WORDS
idx2word = ['<PAD>','<START>', '<END>']
for plot in plotsList:

    words = plot.split(' ')

    b=True
    while b:
        if('' in words): 
            words.remove('')
        else: b = False

    for word in words:
        if word not in idx2word:
            idx2word.append(word)

word2idx = {}
for word in idx2word:
    word2idx[word] = len(word2idx)

vocabLength = len(idx2word)
print('Vocabulary Size: {}'.format(vocabLength))

Vocabulary Size: 71827


__5 - Preprocess text__

In [10]:
def createInputTarget(words) :
    splitNumber = random.randint(1,len(words)-1)
    encode_tokens, decode_tokens = words[:splitNumber], words[splitNumber:]
    encode_tokens = ' '.join(['<START>'] + encode_tokens + ['<END>'])
    output_tokens = ' '.join(decode_tokens + ['<END>'])
    decode_tokens = ' '.join(['<START>'] + decode_tokens + ['<END>'])
    
    return encode_tokens, output_tokens, decode_tokens

def getWordTokens(sentence):
    #clean tokens
    words = sentence.split(' ')
    words.append('.')
    b=True
    while b:
        if('' in words): 
            words.remove('')
        else: b = False
    
    return words

def checkMaxLength(words):
    
    seq = []
    
    if len(words) > SENTENCES_MAX_LENGTH :
        seq.append(words[:SENTENCES_MAX_LENGTH])
        seq.append(words[SENTENCES_MAX_LENGTH:])
        while len(seq[-1]) > SENTENCES_MAX_LENGTH:
            tmp = seq[-1]
            seq[-1] = tmp[:SENTENCES_MAX_LENGTH]
            seq.append(tmp[SENTENCES_MAX_LENGTH:])
    else : 
        seq.append(words)

    return seq

# EXTRACT ENCODER & DECODER INPUT SENTENCES
inputSentences = []
targetSentences = []
outputSentences = []

for plot in trainPlotsList :
    sentences = plot.split('.')
    last = None 
    
    for idx, s in enumerate(sentences):
        words = getWordTokens(s)
        if(len(words) > 2):
            
            seq = checkMaxLength(words)
            
            if(last != None):
                encode_tokens, decode_tokens = last, seq[0]
                encode_tokens = ' '.join(['<START>'] + encode_tokens + ['<END>'])
                output_tokens = ' '.join(decode_tokens + ['<END>'])
                decode_tokens = ' '.join(['<START>'] + decode_tokens + ['<END>'])
                inputSentences.append(encode_tokens)
                targetSentences.append(decode_tokens)
                outputSentences.append(output_tokens)
            
            last = seq[-1]
            
            for s1 in seq:
                if(len(s1) > 2):
                    encode_tokens, output_tokens, decode_tokens = createInputTarget(s1)
                    inputSentences.append(encode_tokens)
                    targetSentences.append(decode_tokens)
                    outputSentences.append(output_tokens)
            


numSamples = len(inputSentences)
print('Num samples: {}'.format(numSamples))
stepsPerEpoch = numSamples//BATCH_SIZE
print('StepsPerEpoch: {}'.format(stepsPerEpoch))

print('EXAMPLE OF PREPROCESSING: ')
for inp, outp in zip(inputSentences[:10],targetSentences[:10]):
    print('INPUT: {}'.format(inp))
    print('OUTPUT: {}'.format(outp))

# WRITE DATASET TO TXT  
train_dataset = []

print("Creating dataset to feed Model . . . ")
dirname = os.path.abspath('')
filePath = os.path.join(dirname, os.path.join(dirname, 'preprocessed/dataset_{}_{}_{}_{}_{}_{}.csv'.format(
EPOCHS, 
SENTENCES_MAX_LENGTH, 
BATCH_SIZE, 
EMBEDDING_DIM,
HIDDEN_DIM,
NUM_TRAIN_PLOTS)))
if os.path.exists(filePath):
    os.remove(filePath) 

d= {'input_encoder' : inputSentences, 'input_decoder' :targetSentences, 'output_decoder':outputSentences }
df = pd.DataFrame(data=d) 
df = shuffle(df)
df.to_csv(filePath, index=False)

print("Dataset printed on CSV.")

Num samples: 350543
StepsPerEpoch: 2738
EXAMPLE OF PREPROCESSING: 
INPUT: <START> a bartender is working <END>
OUTPUT: <START> at a saloon serving drinks to customers . <END>
INPUT: <START> a bartender is working at a saloon serving drinks to customers . <END>
OUTPUT: <START> after he fills a stereotypically irish man bucket with beer carrie nation and her followers burst inside . <END>
INPUT: <START> after he fills a stereotypically irish man bucket with beer carrie nation and her followers burst inside <END>
OUTPUT: <START> . <END>
INPUT: <START> after he fills a stereotypically irish man bucket with beer carrie nation and her followers burst inside . <END>
OUTPUT: <START> they assault the irish man pulling his hat over his eyes and then dumping the beer over his head . <END>
INPUT: <START> they assault the irish man <END>
OUTPUT: <START> pulling his hat over his eyes and then dumping the beer over his head . <END>
INPUT: <START> they assault the irish man pulling his hat over his ey

__6 - Define DataGenerator__

In [11]:
def data_generator(word_2_idx, num_samples, max_length, vocab_length, batch_size=BATCH_SIZE):
    '''
    '''
    dirname = os.path.abspath('')
    filePath = os.path.join(dirname, 'preprocessed/dataset_{}_{}_{}_{}_{}_{}.csv'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS))
    df = pd.read_csv(filePath)
    
    counter = 0

    while True:

        index = 0
        for idx, row in df.iterrows():
        
            if index >= batch_size:
                break

            if counter >= numSamples:
                break

            encoderTokens = row['input_encoder'].split(' ')
            decoderTokens = row['input_decoder'].split(' ')
            outputTokens = row['output_decoder'].split(' ')
            
            b = True
            while b:
                if('' in encoderTokens): encoderTokens.remove('')
                else: b = False

            b = True
            while b:
                if('' in decoderTokens): decoderTokens.remove('')
                else: b = False

            encoderInputData = np.zeros((1, max_length + 2), dtype='int')
            decoderInputData = np.zeros((1, max_length + 2), dtype='int')
            decoderTargetData = np.zeros((1, max_length + 2, 1),dtype='int')
            
            for t, word in enumerate(encoderTokens):
                encoderInputData[0, t] = word_2_idx[word]
            for t, word in enumerate(decoderTokens):
                decoderInputData[0, t] = word_2_idx[word]
            for t, word in enumerate(outputTokens):
                # decoderTargetData is ahead of decoderInputData by one timestep
                decoderTargetData[0, t, 0] = word_2_idx[word]
                
            df.drop(df.index[[idx]])
            index = index + 1
            counter = counter + 1
            
            yield([encoderInputData,decoderInputData], decoderTargetData)

__7 - Train the model__

In [16]:
dirname = os.path.abspath('')

transformerModelPath = os.path.join(dirname, 'models/tr_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS)
)

# Build the model
model = get_model(
    token_num=len(word2idx),
    embed_dim=EMBEDDING_DIM,
    encoder_num=1,
    decoder_num=1,
    head_num=16,
    hidden_dim=HIDDEN_DIM,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((len(word2idx), EMBEDDING_DIM)),
)

model.compile(
    optimizer= keras.optimizers.Adam(),
    loss= keras.losses.sparse_categorical_crossentropy,
    metrics={},
    # Note: There is a bug in keras versions 2.2.3 and 2.2.4 which causes "Incompatible shapes" error, if any type of accuracy metric is used along with sparse_categorical_crossentropy. Use keras<=2.2.2 to use get validation accuracy.
)

model.summary()

if not os.path.exists(transformerModelPath):

    trainGen = data_generator(
            word_2_idx=word2idx,
            num_samples=numSamples,
            max_length=SENTENCES_MAX_LENGTH, 
            vocab_length=vocabLength
        )

    # Train the model
    model.fit_generator(
            trainGen,
            steps_per_epoch=stepsPerEpoch,
            epochs=EPOCHS
            )
    
    model.save_weights(transformerModelPath)     

else : 
    print('Model already trained')



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Token-Embedding (EmbeddingRet)  [(None, None, 128),  9193856     Encoder-Input[0][0]              
                                                                 Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, None, 128)    0           Token-Embedding[0][0]            
__________

__8 - Generate sentences__

In [ ]:
dirname = os.path.abspath('')

transformerModelPath = os.path.join(dirname, 'models/tr_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS)
)

# Build the model
model = get_model(
    token_num=len(word2idx),
    embed_dim=EMBEDDING_DIM,
    encoder_num=1,
    decoder_num=1,
    head_num=16,
    hidden_dim=HIDDEN_DIM,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((len(word2idx), EMBEDDING_DIM)),
)

model.compile(
    optimizer= keras.optimizers.Adam(),
    loss= keras.losses.sparse_categorical_crossentropy,
    metrics={},
    # Note: There is a bug in keras versions 2.2.3 and 2.2.4 which causes "Incompatible shapes" error, if any type of accuracy metric is used along with sparse_categorical_crossentropy. Use keras<=2.2.2 to use get validation accuracy.
)

model.load_weights(transformerModelPath)

sentences = [
    'On a rainy day', 
    'On a rainy day a strange man' , 
    'On a rainy day a strange man was walking',
    'On a rainy day a strange man was walking along the road' ,
    'During the election, in which he is elected governor, she is watching him',
    'Emily, who has no real job, is attracted to Ned'
]

decoded_sentences = []
    
for s in sentences:

    print('Generating from: {}'.format(s))
    encoderTokens = []
    s = clean(s)
    encoderwords = s.split(' ')
    for w in encoderwords:
        encoderTokens.append(word2idx[w])
    encoderTokens = [word2idx['<START>']] + encoderTokens + [word2idx['<END>']]
    encoderInputData = np.zeros((1, SENTENCES_MAX_LENGTH + 2), dtype='int64')

    decoded = decode(
    model,
    encoderTokens,
    start_token=word2idx['<START>'],
    end_token=word2idx['<END>'],
    pad_token=word2idx['<PAD>'],
    max_len=SENTENCES_MAX_LENGTH,
    )

    decodedPhrase = ''
    for x in decoded:
        decodedPhrase = decodedPhrase + ' ' + idx2word[x]

    decoded_sentences.append(decodedPhrase)
    print('Generated: {}'.format(decodedPhrase))
    
resultsModelPatht = os.path.join(dirname, 'output_data/output_{}_{}_{}_{}_{}_{}.csv'.format(
    EPOCHS, 
    SENTENCES_MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_TRAIN_PLOTS)
)

dict ={
    'phrase' : sentences,
    'generated' : decoded_sentences
}

sentiment_df = pd.DataFrame.from_dict(dict)
sentiment_df.to_csv(resultsModelPatht, index=False)